In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 512
epoch = 1000/10

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
x_train_all, x_test_all = x_train_all/255, x_test_all/255

In [6]:
FGSM = []
PGD_10 = []
PGD_100 = []
for base_model_layer in range(0, 51, 10):
    FGSM.append(onp.load("%s_FGSM_%d_layer.npy"%(DATASET, base_model_layer)))
    PGD_10.append(onp.load("%s_PGD_10_%d_layer.npy"%(DATASET, base_model_layer)))
    PGD_100.append(onp.load("%s_PGD_10_%d_layer.npy"%(DATASET, base_model_layer)))

In [7]:
y_test = onp.load("%s_y_test.npy"%(DATASET))

In [8]:
x_test = onp.load("%s_x_test.npy"%(DATASET))

In [9]:
x_test.shape

(2048, 3072)

In [10]:
class Model(tf.keras.Model):
    def __init__(self, layers):
        super(Model, self).__init__()
        self.layers_num = layers
        self.dense = [tf.keras.layers.Dense(512) for i in range(self.layers_num)]
        self.out = tf.keras.layers.Dense(10)
        
    def call(self, inputs):
        x = inputs
        for f in self.dense:
            x = f(x)
            x = tf.math.erf(x)
            
        out = self.out(x)
        return out

In [ ]:
loss_curve = []
acc_curve  = []

loss_curve_FGSM = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_FGSM  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_10 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_10  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_100 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_100  = {idx*10+1: [] for idx in range(len(FGSM))}

for i in range(run):
    model = Model(10)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    for j in range(int(epoch)):
        model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=10)
        # CLEAN
        r = model.evaluate(x_test, y_test, verbose=0)
        loss_curve.append(r[0])
        acc_curve.append(r[1])
        
        for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
                
            r = model.evaluate(F,    y_test, verbose=0)
            print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_FGSM[idx*10+1].append(r[0])
            acc_curve_FGSM[idx*10+1].append(r[1])
        
            r = model.evaluate(_10,  y_test, verbose=0)
            print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_10[idx*10+1].append(r[0])
            acc_curve_PGD_10[idx*10+1].append(r[1])
            
            r = model.evaluate(_100, y_test, verbose=0)
            print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_100[idx*10+1].append(r[0])
            acc_curve_PGD_100[idx*10+1].append(r[1])
            

Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 3s 69us/sample - loss: 2.0769 - accuracy: 0.2370
Epoch 2/10
50000/50000 [==============================] - 2s 36us/sample - loss: 1.8931 - accuracy: 0.3147
Epoch 3/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.8720 - accuracy: 0.3223
Epoch 4/10
50000/50000 [==============================] - 1s 13us/sample - loss: 1.8235 - accuracy: 0.3387
Epoch 5/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.8261 - accuracy: 0.3412
Epoch 6/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.7948 - accuracy: 0.3552
Epoch 7/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.7560 - accuracy: 0.3667
Epoch 8/10
50000/50000 [==============================] - 1s 13us/sample - loss: 1.7666 - accuracy: 0.3668
Epoch 9/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.7324 - accuracy: 0.3777
Epoch 10/10
50

PGD-100 1,  loss: 2.128, acc: 0.306
FGSM    11,  loss: 2.698, acc: 0.195
PGD-10  11,  loss: 2.672, acc: 0.200
PGD-100 11,  loss: 2.672, acc: 0.200
FGSM    21,  loss: 2.869, acc: 0.166
PGD-10  21,  loss: 2.844, acc: 0.172
PGD-100 21,  loss: 2.844, acc: 0.172
FGSM    31,  loss: 2.973, acc: 0.149
PGD-10  31,  loss: 2.948, acc: 0.152
PGD-100 31,  loss: 2.948, acc: 0.152
FGSM    41,  loss: 3.046, acc: 0.138
PGD-10  41,  loss: 3.022, acc: 0.141
PGD-100 41,  loss: 3.022, acc: 0.141
FGSM    51,  loss: 3.104, acc: 0.126
PGD-10  51,  loss: 3.080, acc: 0.133
PGD-100 51,  loss: 3.080, acc: 0.133
Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 13us/sample - loss: 1.2347 - accuracy: 0.5542
Epoch 2/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.2458 - accuracy: 0.5524
Epoch 3/10
50000/50000 [==============================] - 1s 12us/sample - loss: 1.2206 - accuracy: 0.5615
Epoch 4/10
50000/50000 [==============================] - 1s 12us

50000/50000 [==============================] - 1s 11us/sample - loss: 0.0474 - accuracy: 0.9958
Epoch 7/10
50000/50000 [==============================] - 1s 12us/sample - loss: 0.0230 - accuracy: 0.9998
Epoch 8/10
50000/50000 [==============================] - 1s 12us/sample - loss: 0.0151 - accuracy: 0.9999
Epoch 9/10
50000/50000 [==============================] - 1s 12us/sample - loss: 0.0107 - accuracy: 1.0000
Epoch 10/10
50000/50000 [==============================] - 1s 12us/sample - loss: 0.0092 - accuracy: 1.0000
FGSM    1,  loss: 5.170, acc: 0.286
PGD-10  1,  loss: 5.166, acc: 0.290
PGD-100 1,  loss: 5.166, acc: 0.290
FGSM    11,  loss: 6.750, acc: 0.181
PGD-10  11,  loss: 6.699, acc: 0.182
PGD-100 11,  loss: 6.699, acc: 0.182
FGSM    21,  loss: 7.162, acc: 0.158
PGD-10  21,  loss: 7.098, acc: 0.157
PGD-100 21,  loss: 7.098, acc: 0.157
FGSM    31,  loss: 7.432, acc: 0.142
PGD-10  31,  loss: 7.359, acc: 0.141
PGD-100 31,  loss: 7.359, acc: 0.141
FGSM    41,  loss: 7.633, acc: 0.1

PGD-10  51,  loss: 11.222, acc: 0.113
PGD-100 51,  loss: 11.222, acc: 0.113
Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 12us/sample - loss: 3.6407e-04 - accuracy: 1.0000
Epoch 2/10
50000/50000 [==============================] - 1s 12us/sample - loss: 3.4101e-04 - accuracy: 1.0000
Epoch 3/10
50000/50000 [==============================] - 1s 12us/sample - loss: 3.1792e-04 - accuracy: 1.0000
Epoch 4/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.9931e-04 - accuracy: 1.0000
Epoch 5/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.7859e-04 - accuracy: 1.0000
Epoch 6/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.6219e-04 - accuracy: 1.0000
Epoch 7/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.4499e-04 - accuracy: 1.0000
Epoch 8/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.3002e-04 - accuracy: 1.0000
Epoch 9/10
50

50000/50000 [==============================] - 1s 12us/sample - loss: 2.5117e-05 - accuracy: 1.0000
Epoch 9/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.4043e-05 - accuracy: 1.0000
Epoch 10/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3284e-05 - accuracy: 1.0000
FGSM    1,  loss: 9.711, acc: 0.265
PGD-10  1,  loss: 9.704, acc: 0.264
PGD-100 1,  loss: 9.704, acc: 0.264
FGSM    11,  loss: 12.738, acc: 0.154
PGD-10  11,  loss: 12.658, acc: 0.152
PGD-100 11,  loss: 12.658, acc: 0.152
FGSM    21,  loss: 13.639, acc: 0.135
PGD-10  21,  loss: 13.543, acc: 0.132
PGD-100 21,  loss: 13.543, acc: 0.132
FGSM    31,  loss: 14.210, acc: 0.116
PGD-10  31,  loss: 14.144, acc: 0.118
PGD-100 31,  loss: 14.144, acc: 0.118
FGSM    41,  loss: 14.639, acc: 0.105
PGD-10  41,  loss: 14.580, acc: 0.115
PGD-100 41,  loss: 14.580, acc: 0.115
FGSM    51,  loss: 14.972, acc: 0.100
PGD-10  51,  loss: 14.919, acc: 0.107
PGD-100 51,  loss: 14.919, acc: 0.107

...

PGD-100 1,  loss: 10.729, acc: 0.264
FGSM    11,  loss: 13.960, acc: 0.175
PGD-10  11,  loss: 13.861, acc: 0.177
PGD-100 11,  loss: 13.861, acc: 0.177
FGSM    21,  loss: 14.892, acc: 0.154
PGD-10  21,  loss: 14.785, acc: 0.154
PGD-100 21,  loss: 14.785, acc: 0.154
FGSM    31,  loss: 15.505, acc: 0.141
PGD-10  31,  loss: 15.382, acc: 0.143
PGD-100 31,  loss: 15.382, acc: 0.143
FGSM    41,  loss: 15.896, acc: 0.132
PGD-10  41,  loss: 15.837, acc: 0.134
PGD-100 41,  loss: 15.837, acc: 0.134
FGSM    51,  loss: 16.216, acc: 0.123
PGD-10  51,  loss: 16.198, acc: 0.127
PGD-100 51,  loss: 16.198, acc: 0.127
Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 12us/sample - loss: 8.5627e-06 - accuracy: 1.0000
Epoch 2/10
50000/50000 [==============================] - 1s 12us/sample - loss: 8.3403e-06 - accuracy: 1.0000
Epoch 3/10
50000/50000 [==============================] - 1s 11us/sample - loss: 8.1809e-06 - accuracy: 1.0000
Epoch 4/10
50000/50000 [=============

50000/50000 [==============================] - 1s 12us/sample - loss: 2.3133 - accuracy: 0.0989
Epoch 6/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3160 - accuracy: 0.0979
Epoch 7/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3157 - accuracy: 0.1016
Epoch 8/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3142 - accuracy: 0.1000
Epoch 9/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3145 - accuracy: 0.0993
Epoch 10/10
50000/50000 [==============================] - 1s 17us/sample - loss: 2.3139 - accuracy: 0.0991
FGSM    1,  loss: 2.319, acc: 0.097
PGD-10  1,  loss: 2.319, acc: 0.097
PGD-100 1,  loss: 2.319, acc: 0.097
FGSM    11,  loss: 2.319, acc: 0.097
PGD-10  11,  loss: 2.319, acc: 0.097
PGD-100 11,  loss: 2.319, acc: 0.097
FGSM    21,  loss: 2.319, acc: 0.097
PGD-10  21,  loss: 2.319, acc: 0.097
PGD-100 21,  loss: 2.319, acc: 0.097
FGSM    31,  loss: 2.319, acc: 0.097
P

FGSM    51,  loss: 2.312, acc: 0.101
PGD-10  51,  loss: 2.312, acc: 0.101
PGD-100 51,  loss: 2.312, acc: 0.101
Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3160 - accuracy: 0.1002
Epoch 2/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3163 - accuracy: 0.0992
Epoch 3/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3121 - accuracy: 0.0974
Epoch 4/10
50000/50000 [==============================] - 1s 11us/sample - loss: 2.3158 - accuracy: 0.0993
Epoch 5/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3155 - accuracy: 0.0971
Epoch 6/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3171 - accuracy: 0.0963
Epoch 7/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3198 - accuracy: 0.1013
Epoch 8/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3186 - accuracy: 0.0989
Epoch 9/10

50000/50000 [==============================] - 1s 12us/sample - loss: 2.3160 - accuracy: 0.1013
FGSM    1,  loss: 2.316, acc: 0.095
PGD-10  1,  loss: 2.316, acc: 0.095
PGD-100 1,  loss: 2.316, acc: 0.095
FGSM    11,  loss: 2.316, acc: 0.095
PGD-10  11,  loss: 2.316, acc: 0.095
PGD-100 11,  loss: 2.316, acc: 0.095
FGSM    21,  loss: 2.316, acc: 0.095
PGD-10  21,  loss: 2.316, acc: 0.095
PGD-100 21,  loss: 2.316, acc: 0.095
FGSM    31,  loss: 2.316, acc: 0.095
PGD-10  31,  loss: 2.316, acc: 0.095
PGD-100 31,  loss: 2.316, acc: 0.095
FGSM    41,  loss: 2.316, acc: 0.095
PGD-10  41,  loss: 2.316, acc: 0.095
PGD-100 41,  loss: 2.316, acc: 0.095
FGSM    51,  loss: 2.316, acc: 0.095
PGD-10  51,  loss: 2.316, acc: 0.095
PGD-100 51,  loss: 2.316, acc: 0.095
Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3158 - accuracy: 0.1003
Epoch 2/10
50000/50000 [==============================] - 1s 10us/sample - loss: 2.3146 - accuracy: 0.1002
Epoc

50000/50000 [==============================] - 1s 12us/sample - loss: 2.3134 - accuracy: 0.0994
Epoch 5/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3135 - accuracy: 0.1002
Epoch 6/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.3164 - accuracy: 0.1006
Epoch 7/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3169 - accuracy: 0.1027
Epoch 8/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.3119 - accuracy: 0.0995
Epoch 9/10
50000/50000 [==============================] - 1s 12us/sample - loss: 2.3168 - accuracy: 0.0986
Epoch 10/10
50000/50000 [==============================] - 1s 13us/sample - loss: 2.3137 - accuracy: 0.0996
FGSM    1,  loss: 2.306, acc: 0.109
PGD-10  1,  loss: 2.306, acc: 0.109
PGD-100 1,  loss: 2.306, acc: 0.109
FGSM    11,  loss: 2.306, acc: 0.109
PGD-10  11,  loss: 2.306, acc: 0.109
PGD-100 11,  loss: 2.306, acc: 0.109
FGSM    21,  loss: 2.306, acc: 0.109
PGD-1

In [ ]:
loss_curve_FGSM

In [ ]:
for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
    r = model.evaluate(F,    y_test, verbose=0)
    print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_10,  y_test, verbose=0)
    print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_100, y_test, verbose=0)
    print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))